In [ ]:
# default_exp app

# Understat-DB CLI

> CLI for creating a soccer database with data from understat.com

In [ ]:
#hide
from nbdev.showdoc import *

Initialise the app with Typer

In [ ]:
#export
import os
import dotenv
import playhouse.postgres_ext
import typer

import understatdb.db


app = typer.Typer()

## Hello, world

An example method

In [ ]:
#export
@app.command()
def hello(name: str):
    typer.echo(f'Hello, {name}!')

Test the app using Typer's in-build CLI-runner

In [ ]:
import typer.testing

runner = typer.testing.CliRunner()
result = runner.invoke(app, ['World'])
assert result.exit_code == 0
assert 'Hello, World!' in result.output

## Migrate

Migrating the database

In [ ]:
#export


@app.command()
def migrate(interactive: bool=True):
    dotenv.load_dotenv()
    
    # Load database config from environment
    postgres_db = playhouse.postgres_ext.PostgresqlExtDatabase(
        host=os.environ['DB_HOST'],
        user=os.environ['DB_USER'],
        password=os.environ['DB_PASS'],
        database=os.environ['DB_NAME'],
        port=os.environ['DB_PORT'],
    )

    # Configure proxy database to use configured postgres
    typer.secho('Initialising database connection...', fg=typer.colors.BRIGHT_BLACK)
    understatdb.db.DB.initialize(postgres_db)
    
    # Migrate database tables
    typer.secho('Migrating database tables...', fg=typer.colors.BRIGHT_BLACK)
    understatdb.db.DB.evolve(
        ignore_tables=understatdb.db.EVOLVE_IGNORE_TABLES,
        interactive=interactive
    )
    typer.secho('Done!', fg=typer.colors.GREEN, bold=True)

In [ ]:
dotenv.load_dotenv()

True

---

Finally, make the app executable from the command line

In [ ]:
#export

# Try/except block seems to be the 'canonical'
# way to export __name__ == __main__ in nbdev.
# By excepting an ImportError, we don't have to
# include nbdev as a runtime dependency (only a
# development dependency).
# 
# See: 
#  * https://pete88b.github.io/fastpages/nbdev/fastai/jupyter/2020/07/24/nbdev-deep-dive.html#Export-a-if-__name__-==-
#  * https://forums.fast.ai/t/nbdev-is-there-a-way-to-export-a-if-name-main-clause/73050/3
try:
    from nbdev.imports import IN_NOTEBOOK
except ImportError: 
    IN_NOTEBOOK = False

if __name__ == '__main__' and not IN_NOTEBOOK:
    app()